In [2]:
# implementar:
# verificar elementos carregarem em vez de time.sleep
# verificar o tipo de página na antaq(inexistente, alterável e protocolado)
# for para fazer múltiplos valores, esperar depois que o protocolo for enviado
# bypass captcha

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from selenium.webdriver.common.keys import Keys

from selenium.webdriver.support.ui import Select

import time

imolist = [
    '9508469',
]

email = 'dcampos@brssz.com'

for imo in imolist:
    servico = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=servico)

    # entra no site e aguarda carregar
    driver.get("https://www.marinetraffic.com/en/global-search/")
    # espera até aviso aparecer para continuar e aceita os cookies
    element = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH,'//*[@id="qc-cmp2-ui"]/div[2]/div/button[2]')))
    driver.find_element('xpath', '//*[@id="qc-cmp2-ui"]/div[2]/div/button[2]').click()
    time.sleep(2)
    
    # digita IMO e pesquisa
    driver.find_element('xpath', "//*[@id='app']/div/div[2]/div[2]/header/div/div[1]/div/div[2]/div/div/input").send_keys(imo)
    driver.find_element('xpath', "//*[@id='app']/div/div[2]/div[2]/header/div/div[1]/div/div[2]/div/div/input").send_keys(Keys.RETURN)
    # clica no primeiro resultado
    
    #element = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH,"//*[@id='app']/div/div[2]/div[2]/div/div/div[1]/div[1]/div/a/h5")))
    time.sleep(2)
    driver.find_element('xpath',"//*[@id='app']/div/div[2]/div[2]/div/div/a[1]/div").click()
    time.sleep(2)

    # scroll até a área onde os dados estão localizados
    driver.execute_script("scroll(0, 1600)")
    element = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH,'//*[@id="General-content"]/div')))

    time.sleep(4)
    # coleta dos dados
    #element = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH,'//*[@id="shipName"]/b')))
    nome = driver.find_element('xpath', '//*[@id="shipName"]/b').text

    callSign = driver.find_element('xpath', '//*[@id="callSign"]/b').text
    bandeira = driver.find_element('xpath', '//*[@id="flag"]/b').text
    tipo = driver.find_element('xpath', '//*[@id="shipTypeSpecific"]/b').text
    ano = driver.find_element('xpath', '//*[@id="yearBuilt"]/b').text
    arqueacao = driver.find_element('xpath', '//*[@id="grossTonnage"]/b').text
    dwt = driver.find_element('xpath', '//*[@id="summerDwt"]/b').text

    # tratamento dos dados
    bandeira = bandeira[-3] + bandeira[-2] # pegando apenas a sigla da bandeira
    arqueacao += '00'
    dwt = dwt.replace(' t', '')
    dwt += '00'

    # separando cada caractere das strings arqueacao e dwt devido a um bug no formulário da ANTAQ
    arqueacao = (" ".join(arqueacao))
    dwt = (" ".join(dwt))

    print(f"Nome: {nome},\nIMO: {imo},\nCallSign: {callSign},\nBandeira: {bandeira},\nTipo: {tipo},\nAno: {ano},\nArqueação: {arqueacao},\nDWT: {dwt}")


    # SITE ANTAQ

    '''
    # entra no site da antaq e preenche imo
    driver.get("https://web3.antaq.gov.br/SAMA/Embarcacao/Consultar.aspx")
    time.sleep(5)
    driver.find_element('xpath', '//*[@id="txtIMO"]').click()
    driver.find_element('xpath', '//*[@id="txtIMO"]').send_keys(imo)
    driver.find_element('xpath', '//*[@id="txtIMO"]').send_keys(Keys.RETURN)

    # verifica se o popup de navio já cadastrado aparece
    if (len(driver.find_elements('xpath', '//*[@id="msgCadastrar"]'))):
        driver.find_element('xpath', '//*[@id="btnCadastrarSim"]').click()

    time.sleep(1)
    '''
    
    driver.get(f"https://web3.antaq.gov.br/SAMA/Embarcacao/Cadastro.aspx?Novo={imo}")
    time.sleep(5)
    
    
    # preenchimento de dados
    driver.find_element('xpath', '//*[@id="txtNome"]').send_keys(nome)
    driver.find_element('xpath', '//*[@id="txtIRIN"]').send_keys(callSign)
    driver.find_element('xpath', '//*[@id="ddlBandeira"]').click()
    select = Select(driver.find_element('xpath','//*[@id="ddlBandeira"]')).select_by_value(bandeira)

    # verifica o tipo de navio e carga
    driver.find_element('xpath', '//*[@id="ddlTipoEmbarcacao"]').click()
    if (tipo == 'Bulk Carrier'):
        driver.find_element('xpath', '//*[@id="ddlTipoEmbarcacao"]/option[19]').click()
        driver.find_element('xpath', '//*[@id="vncTipoCarga"]/select[1]/option[9]').click()
        driver.find_element('xpath', '//*[@id="vncTipoCarga"]/div/input[3]').click()
    elif (tipo == 'General Cargo'):
        driver.find_element('xpath', '//*[@id="ddlTipoEmbarcacao"]/option[10]').click()
        driver.find_element('xpath', '//*[@id="vncTipoCarga"]/select[1]/option[4]').click()
        driver.find_element('xpath', '//*[@id="vncTipoCarga"]/div/input[3]').click()
    elif (tipo == 'Tanker'):
        driver.find_element('xpath', '//*[@id="ddlTipoEmbarcacao"]/option[19]').click()
        time.sleep(1)
        driver.find_element('xpath', '//*[@id="ddlClasse"]/option[2]').click()
        driver.find_element('xpath', '//*[@id="vncTipoCarga"]/select[1]/option[11]').click()
        driver.find_element('xpath', '//*[@id="vncTipoCarga"]/div/input[3]').click()
    elif (tipo == 'Oil Tanker' or tipo == 'Crude Oil Tanker'):
        driver.find_element('xpath', '//*[@id="ddlTipoEmbarcacao"]/option[31]').click()
        # opção de petróleo não aparecendo
        driver.find_element('xpath', '//*[@id="vncTipoCarga"]/div/input[3]').click()
    elif (tipo == 'Passenger Ship'):
        driver.find_element('xpath', '//*[@id="ddlTipoEmbarcacao"]/option[28]').click() # marcado como opção passageiros, confirmar depois
        driver.find_element('xpath', '//*[@id="vncTipoCarga"]/select[1]/option[15]').click()
        driver.find_element('xpath', '//*[@id="vncTipoCarga"]/div/input[3]').click()


    # preenchendo dados de ano, arqueação e dwt
    driver.find_element('xpath', '//*[@id="txtAno"]').send_keys(ano)
    driver.find_element('xpath', '//*[@id="txtArqueacao"]').send_keys(arqueacao)
    driver.find_element('xpath', '//*[@id="txtTPB"]').send_keys(dwt)

    # selecionando checkbox de cabotagem e longo curso
    driver.find_element('xpath', '//*[@id="chkCabotagem"]').click()
    time.sleep(2)
    driver.find_element('xpath', '//*[@id="chkLongoCurso"]').click()

    # inserindo email
    driver.find_element('xpath', '//*[@id="txtEmail"]').send_keys(email)
    driver.find_element('xpath', '//*[@id="txtConfirmarEmail"]').send_keys(email)

    driver.find_element('xpath', '//*[@id="rdbSDP"]').click()

    callSign = ''
    bandeira = ''
    tipo = ''
    ano = ''
    arqueacao = ''
    dwt = ''
    
    keyword = input("enter a character or press enter to continue")

# apagar "Não informado" do input de nome do navio
# adicionar roll on roll off
# adicionar container ship
# adicionar exceção pra pular para o próximo caso navio já esteja cadastrado

WebDriverException: Message: unknown error: cannot determine loading status
from disconnected: received Inspector.detached event
  (Session info: chrome=105.0.5195.102)
Stacktrace:
Backtrace:
	Ordinal0 [0x0076DF13+2219795]
	Ordinal0 [0x00702841+1779777]
	Ordinal0 [0x0061423D+803389]
	Ordinal0 [0x00607DF8+753144]
	Ordinal0 [0x00607AF6+752374]
	Ordinal0 [0x0060706A+749674]
	Ordinal0 [0x00605F94+745364]
	Ordinal0 [0x00606528+746792]
	Ordinal0 [0x0060F65E+783966]
	Ordinal0 [0x0061A695+829077]
	Ordinal0 [0x0061DA60+842336]
	Ordinal0 [0x006067F6+747510]
	Ordinal0 [0x0061A28F+828047]
	Ordinal0 [0x0066F0BA+1175738]
	Ordinal0 [0x0065E616+1107478]
	Ordinal0 [0x00637F89+950153]
	Ordinal0 [0x00638F56+954198]
	GetHandleVerifier [0x00A62CB2+3040210]
	GetHandleVerifier [0x00A52BB4+2974420]
	GetHandleVerifier [0x00806A0A+565546]
	GetHandleVerifier [0x00805680+560544]
	Ordinal0 [0x00709A5C+1808988]
	Ordinal0 [0x0070E3A8+1827752]
	Ordinal0 [0x0070E495+1827989]
	Ordinal0 [0x007180A4+1867940]
	BaseThreadI

In [3]:
# checa cada imo para ver a situação da homologação

imolist = [
    "9725471",
    "9651101",
    "9442469",
    "9311177",
    "9460760",
    "9573634",
    "9576961",
    "9285146",
    "9935466",
    "9900095",
    "9705354",
    "9289025",
    "9713935",
    "9193317",
    "9374351",
    "9457878",
    "9569243",
    "9675274",
    "9467249",
    "9425186",
    "9494486",
    "9491587",
    "9442392",
    "9649299",
    "9642215",
    "9679799",
    "9600607",
    "9604847",
    "9887619",
    "9734185",
    "9298533",
    "9717060",
    "9646663",
    "9300556",
    "9705299",
    "9139268",
]

servico = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=servico)

for imo in imolist:
    #driver.get(f"https://web3.antaq.gov.br/SAMA/Embarcacao/Cadastro.aspx?Novo={imo}")
    driver.get("https://web3.antaq.gov.br/SAMA/Embarcacao/Consultar.aspx")
    time.sleep(2)
    driver.find_element('xpath', '//*[@id="txtIMO"]').click()
    driver.find_element('xpath', '//*[@id="txtIMO"]').send_keys(imo)
    driver.find_element('xpath', '//*[@id="txtIMO"]').send_keys(Keys.RETURN)

    # verifica se o popup de navio já cadastrado aparece
    if (len(driver.find_elements('xpath', '//*[@id="msgCadastrar"]'))):
        driver.find_element('xpath', '//*[@id="btnCadastrarSim"]').click()

    time.sleep(15)